# crawling TransferMarket and selenium for players links

# https://www.transfermarkt.com/

    At this step we will take basic details of each player and the links to the website from which we will obtain new data about the player (with the help of Selenium)

In [38]:
import time      
import os        
import bs4 
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common import keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
urls=["https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/0/galerie/0?ausrichtung=Sturm&spielerposition_id=alle&altersklasse=alle&jahrgang=0&land_id=0&kontinent_id=0&yt0=Show"
     ,"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/0/galerie/0?ausrichtung=Mittelfeld&spielerposition_id=alle&altersklasse=alle&jahrgang=0&land_id=0&kontinent_id=0&yt0=Show"
     ,"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/0/galerie/0?ausrichtung=Abwehr&spielerposition_id=alle&altersklasse=alle&jahrgang=0&land_id=0&kontinent_id=0&yt0=Show"
     ]

These are the links from which we will take the basic values for each player (where you will find your position).
And also our final goal column - the player's value

# crawoling

In [4]:
def getdata(url):
    headers = {"User-Agent":"Mozilla/5.0"}#firefox (with python,we got response 404) 
    html=requests.get(url,headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [5]:
def getnextpage(soup):
    next_page=soup.find("li",attrs={"class":"tm-pagination__list-item tm-pagination__list-item--icon-next-page"}).find('a')['href']
    next_page="https://www.transfermarkt.com"+str(next_page)
    return next_page

In [6]:
def get_table(soup):
    tbl=soup("tbody")[1]
    tbl=tbl("tr",attrs={"class":["even","odd"]})
    return tbl

def get_name(player):
    name=player("td",attrs={"class":"hauptlink"})[0].string
    return name

def get_position(player):
    return player("td")[4].get_text().strip()

def get_age(player):
    return player("td")[5].get_text().strip()

def get_nation(player):
    nation=player("img",attrs={"class":"flaggenrahmen"})[0]
    return nation["alt"]

def get_club(player):
    club=player("td")[7]
    club=club("a")[0]
    return club["title"]

def get_value(player):
    value=player("td",attrs={"class":"rechts hauptlink"})[0]
    return value.get_text().strip()

In [7]:
names=[]
positions=[]
ages=[]
nations=[]
clubs=[]
values=[]

In [8]:
for url in urls:
    for i in range(20):
        soup=getdata(url)
        tbl=get_table(soup)

        for player in tbl:
            names.append(get_name(player))
    
            positions.append(get_position(player))
    
            ages.append(get_age(player))
    
            nations.append(get_nation(player))
    
            clubs.append(get_club(player))

            values.append(get_value(player))
        if i!=19:    
            url=getnextpage(soup)
    

In [9]:
df=pd.DataFrame({'value':values,'name':names,'club':clubs,'nation':nations,'position':positions,'age':ages})
df

,value,name,club,nation,position,age
0,€180.00m,Kylian Mbappé,Paris Saint-Germain,France,Centre-Forward,24
1,€170.00m,Erling Haaland,Manchester City,Norway,Centre-Forward,22
2,€120.00m,Vinicius Junior,Real Madrid,Brazil,Left Winger,22
3,€90.00m,Harry Kane,Tottenham Hotspur,England,Centre-Forward,29
4,€85.00m,Rafael Leão,AC Milan,Portugal,Left Winger,23
...,...,...,...,...,...,...
1495,€5.00m,Federico Gatti,Juventus FC,Italy,Centre-Back,24
1496,€5.00m,Adrià Pedrosa,RCD Espanyol Barcelona,Spain,Left-Back,24
1497,€5.00m,Joaquín Piquerez,Sociedade Esportiva Palmeiras,Uruguay,Left-Back,24
1498,€5.00m,Nicolas Cozza,Montpellier HSC,France,Left-Back,23


In [10]:
df.to_csv("players_transfer_market")

In [11]:
df.dropna()
df.drop_duplicates()

,value,name,club,nation,position,age
0,€180.00m,Kylian Mbappé,Paris Saint-Germain,France,Centre-Forward,24
1,€170.00m,Erling Haaland,Manchester City,Norway,Centre-Forward,22
2,€120.00m,Vinicius Junior,Real Madrid,Brazil,Left Winger,22
3,€90.00m,Harry Kane,Tottenham Hotspur,England,Centre-Forward,29
4,€85.00m,Rafael Leão,AC Milan,Portugal,Left Winger,23
...,...,...,...,...,...,...
1495,€5.00m,Federico Gatti,Juventus FC,Italy,Centre-Back,24
1496,€5.00m,Adrià Pedrosa,RCD Espanyol Barcelona,Spain,Left-Back,24
1497,€5.00m,Joaquín Piquerez,Sociedade Esportiva Palmeiras,Uruguay,Left-Back,24
1498,€5.00m,Nicolas Cozza,Montpellier HSC,France,Left-Back,23


In [12]:
df.describe()

,value,name,club,nation,position,age
count,1500,1500,1500,1500,1500,1500
unique,55,1493,220,85,12,22
top,€6.00m,João Pedro,Liverpool FC,Brazil,Centre-Back,25
freq,120,2,24,148,279,187


# crawoling+ selenium

At this step, with the help of Selenium, we will take the link of each player whose name we took in the previous step.


Selenium will search the site by entering the player's name in the search box and clicking enter.
After that, the player will take out the link so that we can take his details in the next step

In [65]:
def get_player_page(name):
    url='https://fminside.net/players'
    driver = webdriver.Chrome()
    driver.get(url)
    element=driver.find_element(By.XPATH,'//*[@id="sidebar"]/div[2]/form/div[2]/input')
    element.send_keys(name)
    element.send_keys(Keys.ENTER)
    
    time.sleep(10)
    #element=driver.find_element(By.LINK_TEXT,name)
  
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.close()
    try:
        soup=soup("span",attrs={'class':'name'})[0]
        soup=soup.find('a')['href']
        player_url='https://fminside.net'+str(soup)
        return player_url
    except:
        return "N/A"

In [67]:
links=[]
for i in range(len(df)):
    print(i,df['name'][i])
    links.append(get_player_page(df['name'][i]))
    print(links[i])




0 Kylian Mbappé
https://fminside.net/players/3-fm-23/85139014-kylian-mbappe
1 Erling Haaland
https://fminside.net/players/3-fm-23/29179241-erling-haaland
2 Vinicius Junior
https://fminside.net/players/3-fm-23/19302146-vinicius-junior
3 Harry Kane
https://fminside.net/players/3-fm-23/28049320-harry-kane
4 Rafael Leão
https://fminside.net/players/3-fm-23/83111483-rafael-leao
5 Rodrygo
https://fminside.net/players/3-fm-23/19306929-rodrygo
6 Dusan Vlahovic
https://fminside.net/players/3-fm-23/62182055-duan-vlahovic
7 Christopher Nkunku
https://fminside.net/players/3-fm-23/85140301-christopher-nkunku
8 Mohamed Salah
https://fminside.net/players/3-fm-23/98028755-mohamed-salah
9 Antony
https://fminside.net/players/3-fm-23/19338230-antony
10 Luis Díaz
https://fminside.net/players/3-fm-23/76049803-luis-diaz
11 Lautaro Martínez
https://fminside.net/players/3-fm-23/14110660-lautaro-martinez
12 Gabriel Jesus
https://fminside.net/players/3-fm-23/19220266-gabriel-jesus
13 Neymar
https://fminside.net

https://fminside.net/players/3-fm-23/19153072-rafa-silva
107 Domenico Berardi
https://fminside.net/players/3-fm-23/43094553-domenico-berardi
108 Ciro Immobile
https://fminside.net/players/3-fm-23/43010773-ciro-immobile
109 Evanilson
https://fminside.net/players/3-fm-23/19342183-evanilson
110 Luis Sinisterra
https://fminside.net/players/3-fm-23/76047347-luis-sinisterra
111 David Neres
https://fminside.net/players/3-fm-23/19260918-david-neres
112 Ismaïla Sarr
https://fminside.net/players/3-fm-23/48042766-ismaila-sarr
113 Ché Adams
https://fminside.net/players/3-fm-23/90055302-che-adams
114 Nicolas Pépé
https://fminside.net/players/3-fm-23/85136376-nicolas-pepe
115 Saïd Benrahma
https://fminside.net/players/3-fm-23/48036788-said-benrahma
116 Mattia Zaccagni
https://fminside.net/players/3-fm-23/43125343-mattia-zaccagni
117 Viktor Tsygankov
https://fminside.net/players/3-fm-23/71080414-viktor-tsygankov
118 Gabriel Barbosa
https://fminside.net/players/3-fm-23/30035404-gabriel-barbosa
119 End

https://fminside.net/players/3-fm-23/67106568-juanmi
214 Marcos Leonardo
https://fminside.net/players/3-fm-23/19380666-marcos-leonardo
215 Ferran Jutglà
https://fminside.net/players/3-fm-23/67272136-ferran-jutgla
216 Armand Laurienté
https://fminside.net/players/3-fm-23/85145172-armand-lauriente
217 Folarin Balogun
https://fminside.net/players/3-fm-23/28116305-folarin-balogun
218 Brian Brobbey
https://fminside.net/players/3-fm-23/37065620-brian-brobbey
219 Jonas Wind
https://fminside.net/players/3-fm-23/27122565-jonas-wind
220 Justin Kluivert
https://fminside.net/players/3-fm-23/18004457-kevin-de-bruyne
221 Diego Rossi
https://fminside.net/players/3-fm-23/78074592-diego-rossi
222 Ryan Kent
https://fminside.net/players/3-fm-23/28094948-ryan-kent
223 David Brooks
https://fminside.net/players/3-fm-23/28095352-david-brooks
224 Luka Jović
https://fminside.net/players/3-fm-23/62113639-luka-jovic
225 Wout Weghorst
https://fminside.net/players/3-fm-23/37026994-wout-weghorst
226 Gaëtan Laborde


https://fminside.net/players/3-fm-23/5108293-chris-wood
326 Niclas Füllkrug
https://fminside.net/players/3-fm-23/92018043-niclas-fullkrug
327 Ángel Di María
https://fminside.net/players/3-fm-23/14000219-angel-di-maria
328 Giovani
https://fminside.net/players/3-fm-23/14108651-giovani-lo-celso
329 Andreas Schjelderup
https://fminside.net/players/3-fm-23/2000020287-andreas-schjelderup
330 Ricardo Pepi
https://fminside.net/players/3-fm-23/20048604-ricardo-pepi
331 Taty Castellanos
N/A
332 Largie Ramazani
https://fminside.net/players/3-fm-23/28116558-largie-ramazani
333 Jan Thielmann
https://fminside.net/players/3-fm-23/91185080-jan-thielmann
334 Aleksandr Sobolev
N/A
335 Luis Suárez
https://fminside.net/players/3-fm-23/78000335-luis-suarez
336 Jean-Philippe Mateta
https://fminside.net/players/3-fm-23/49034219-jean-philippe-mateta
337 Riccardo Sottil
https://fminside.net/players/3-fm-23/43270713-riccardo-sottil
338 Junya Ito
https://fminside.net/players/3-fm-23/45092815-junya-ito
339 Anders

https://fminside.net/players/3-fm-23/28107919-reiss-nelson
435 Yeferson Soteldo
https://fminside.net/players/3-fm-23/86040729-yeferson-soteldo
436 Toni Martínez
https://fminside.net/players/3-fm-23/67216151-toni-martinez
437 Milot Rashica
https://fminside.net/players/3-fm-23/11024791-milot-rashica
438 Adam Buksa
https://fminside.net/players/3-fm-23/96055358-adam-buksa
439 Cristian Arango
https://fminside.net/players/3-fm-23/76035677-cristian-arango
440 Antonio Sanabria
https://fminside.net/players/3-fm-23/67181404-antonio-sanabria
441 Nicolás López
https://fminside.net/players/3-fm-23/14206238-nicolas-lopez
442 Adnan Januzaj
https://fminside.net/players/3-fm-23/18054092-adnan-januzaj
443 Viktor Claesson
https://fminside.net/players/3-fm-23/93014195-viktor-claesson
444 Suso
https://fminside.net/players/3-fm-23/67085965-suso
445 Joshua King
https://fminside.net/players/3-fm-23/53020695-joshua-king
446 Iago Aspas
https://fminside.net/players/3-fm-23/67010455-iago-aspas
447 Dusan Tadić
htt

...

https://fminside.net/players/3-fm-23/24058104-bartol-franjic
973 Gedson Fernandes
https://fminside.net/players/3-fm-23/83111338-gedson-fernandes
974 Víctor Guzmán
https://fminside.net/players/3-fm-23/51052599-victor-guzman
975 Nahitan Nández
https://fminside.net/players/3-fm-23/78067241-nahitan-nandez
976 Dorukhan Toköz
https://fminside.net/players/3-fm-23/70097267-dorukhan-tokoz
977 Willian Arão
https://fminside.net/players/3-fm-23/19073567-willian-arao
978 Roberto Gagliardini
https://fminside.net/players/3-fm-23/43081390-roberto-gagliardini
979 Ryan Gauld
https://fminside.net/players/3-fm-23/61042729-ryan-gauld
980 Alfred Duncan
https://fminside.net/players/3-fm-23/43084308-alfred-duncan
981 Wataru Endo
https://fminside.net/players/3-fm-23/45030953-wataru-endo
982 Sérgio Oliveira
https://fminside.net/players/3-fm-23/55012099-sergio-oliveira
983 Roberto Pereyra
https://fminside.net/players/3-fm-23/14021900-roberto-pereyra
984 Ismaël Koné
https://fminside.net/players/3-fm-23/2000121309

https://fminside.net/players/3-fm-23/48037291-axel-disasi
1081 Nayef Aguerd
https://fminside.net/players/3-fm-23/13199286-nayef-aguerd
1082 Davinson Sánchez
https://fminside.net/players/3-fm-23/76023027-davinson-sanchez
1083 Kyle Walker-Peters
https://fminside.net/players/3-fm-23/29126231-kyle-walker-peters
1084 Merih Demiral
https://fminside.net/players/3-fm-23/83176849-merih-demiral
1085 Takehiro Tomiyasu
https://fminside.net/players/3-fm-23/45095490-takehiro-tomiyasu
1086 Davide Calabria
https://fminside.net/players/3-fm-23/43123020-davide-calabria
1087 Amir Rrahmani
https://fminside.net/players/3-fm-23/11023165-amir-rrahmani
1088 Tyrone Mings
https://fminside.net/players/3-fm-23/29074544-tyrone-mings
1089 Reinildo Mandava
N/A
1090 Federico Dimarco
https://fminside.net/players/3-fm-23/43143488-federico-dimarco
1091 Giovanni Di Lorenzo
https://fminside.net/players/3-fm-23/43075156-giovanni-di-lorenzo
1092 Gonçalo Inácio
https://fminside.net/players/3-fm-23/83228731-goncalo-inacio
109

https://fminside.net/players/3-fm-23/67215898-igor-zubeldia
1188 Manuel Lazzari
https://fminside.net/players/3-fm-23/18004457-kevin-de-bruyne
1189 Issa Diop
https://fminside.net/players/3-fm-23/85140415-issa-diop
1190 Attila Szalai
https://fminside.net/players/3-fm-23/16147693-attila-szalai
1191 Yeray Álvarez
N/A
1192 Zeki Celik
https://fminside.net/players/3-fm-23/70097248-zeki-celik
1193 Matt Targett
N/A
1194 Abdou Diallo
https://fminside.net/players/3-fm-23/85120721-abdou-diallo
1195 Daniel Amartey
https://fminside.net/players/3-fm-23/93034815-daniel-amartey
1196 Benjamin Henrichs
https://fminside.net/players/3-fm-23/91132680-benjamin-henrichs
1197 Héctor Bellerín
https://fminside.net/players/3-fm-23/28066082-hector-bellerin
1198 Victor Lindelöf
https://fminside.net/players/3-fm-23/93023425-victor-lindelof
1199 Leonardo Spinazzola
https://fminside.net/players/3-fm-23/43036586-leonardo-spinazzola
1200 Stefan de Vrij
https://fminside.net/players/3-fm-23/37001814-stefan-de-vrij
1201 Ky

https://fminside.net/players/3-fm-23/29175064-ethan-ampadu
1295 Nicolò Casale
https://fminside.net/players/3-fm-23/43206174-nicolo-casale
1296 Josip Juranovic
https://fminside.net/players/3-fm-23/24047634-josip-juranovic
1297 Mitchel Bakker
https://fminside.net/players/3-fm-23/37063645-mitchel-bakker
1298 Bright Osayi-Samuel
https://fminside.net/players/3-fm-23/29143882-bright-osayi-samuel
1299 Diogo Leite
https://fminside.net/players/3-fm-23/83111536-diogo-leite
1300 Luca Pellegrini
https://fminside.net/players/3-fm-23/43252226-luca-pellegrini
1301 Japhet Tanganga
https://fminside.net/players/3-fm-23/28106874-japhet-tanganga
1302 Joe Rodon
https://fminside.net/players/3-fm-23/28100477-joe-rodon
1303 Gustavo Gómez
https://fminside.net/players/3-fm-23/79010306-gustavo-gomez
1304 Rick Karsdorp
https://fminside.net/players/3-fm-23/37039938-rick-karsdorp
1305 Vladimir Coufal
https://fminside.net/players/3-fm-23/25038517-vladimir-coufal
1306 Nicolás Tagliafico
https://fminside.net/players/3

https://fminside.net/players/3-fm-23/19172406-paulo-otavio
1401 Romain Saïss
https://fminside.net/players/3-fm-23/85116519-romain-saiss
1402 Marius Wolf
https://fminside.net/players/3-fm-23/91124151-marius-wolf
1403 Valentino Lazaro
https://fminside.net/players/3-fm-23/16067510-valentino-lazaro
1404 Marco Friedl
https://fminside.net/players/3-fm-23/91143703-marco-friedl
1405 Ezgjan Alioski
https://fminside.net/players/3-fm-23/98017001-ezgjan-alioski
1406 Cristiano Biraghi
https://fminside.net/players/3-fm-23/43029905-cristiano-biraghi
1407 Thomas Meunier
https://fminside.net/players/3-fm-23/18043393-thomas-meunier
1408 Mats Hummels
https://fminside.net/players/3-fm-23/35007944-mats-hummels
1409 Borna Barisic
https://fminside.net/players/3-fm-23/24044137-borna-barisic
1410 Anthony Rouault
https://fminside.net/players/3-fm-23/49047179-anthony-rouault
1411 Lucas Piton
https://fminside.net/players/3-fm-23/19380037-lucas-piton
1412 Khellven
https://fminside.net/players/3-fm-23/19369351-khel

In [69]:
df['link']=links
df


,value,name,club,nation,position,age,link
0,€180.00m,Kylian Mbappé,Paris Saint-Germain,France,Centre-Forward,24,https://fminside.net/players/3-fm-23/85139014-...
1,€170.00m,Erling Haaland,Manchester City,Norway,Centre-Forward,22,https://fminside.net/players/3-fm-23/29179241-...
2,€120.00m,Vinicius Junior,Real Madrid,Brazil,Left Winger,22,https://fminside.net/players/3-fm-23/19302146-...
3,€90.00m,Harry Kane,Tottenham Hotspur,England,Centre-Forward,29,https://fminside.net/players/3-fm-23/28049320-...
4,€85.00m,Rafael Leão,AC Milan,Portugal,Left Winger,23,https://fminside.net/players/3-fm-23/83111483-...
...,...,...,...,...,...,...,...
1495,€5.00m,Federico Gatti,Juventus FC,Italy,Centre-Back,24,https://fminside.net/players/3-fm-23/43491384-...
1496,€5.00m,Adrià Pedrosa,RCD Espanyol Barcelona,Spain,Left-Back,24,https://fminside.net/players/3-fm-23/95035991-...
1497,€5.00m,Joaquín Piquerez,Sociedade Esportiva Palmeiras,Uruguay,Left-Back,24,https://fminside.net/players/3-fm-23/78077014-...
1498,€5.00m,Nicolas Cozza,Montpellier HSC,France,Left-Back,23,https://fminside.net/players/3-fm-23/49037761-...


Now we have the link of each of the players to the source from which we will take the relevant details

In [72]:
df.to_csv("players_with_link")
                    

In [73]:
df['link'][0]

'https://fminside.net/players/3-fm-23/85139014-kylian-mbappe'